In [1]:
import pandas as pd
import os
import re

# from ckiptagger import data_utils, WS
# ws = WS("./data",disable_cuda=False) # ckiptagger 模型 

import transformers
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

# from transformers import BertTokenizerFast, AutoModel
# tokenizer = BertTokenizerFast.from_pretrained('ckiplab/albert-base-chinese-ner')
# model = AutoModel.from_pretrained('ckiplab/albert-base-chinese-ner')

In [2]:
# 載入模型，device參數 0 = GPU, -1 = CPU
ckip_ws = CkipWordSegmenter(model="bert-base",device=-1)
ckip_pos = CkipPosTagger(model="bert-base",device=-1)
ckip_ner = CkipNerChunker(model='bert-base',device=-1)

In [3]:
# text = ['這價位真的很不行，全部只有甜點上得了檯面，特別是廁所，難以置信五星級飯店的廁所連免治馬桶都沒有（可能在房間裡面），整間灰灰暗暗，牆壁磁磚顏色還以為是發霉，真的很對不起一起來聚餐的同事，完全不推薦。']
# 
# pos_text = ckip_ws(text)
# print(pos_text)

## 原始評論留言文字檔處理
### 目標
    - 依照分類爬蟲獲取每家餐廳評論文字檔轉換為excel逐行儲存每家餐廳評論
        - excel檔案欄位表
            - 評論留言原內容
            - 評論留言基本處理 (去除表情符號、無意義字符)
            - 斷詞處理內容
            - 停用詞處理內容

### 調整紀錄
    - 留言中有些消費者的文章中有用到,所以在後續將每則評論留言以,作為分隔識別一則一則存入一行一行dataframe中會造成消費者文章中使用的,將本來是同一篇評論分隔開成好幾則造成資料錯誤傳入
        - 本來是將評論.txt做完一些基礎處裡後在輸出為新的評論.txt到新的檔案路徑中，現在改成直接把未處理的評論.txt直接先依照列表形式逐筆存入dataframe中
    - 不少評論留言中出現 👍表情符號，這個表情符號應該有高度正向意思，不過會在資料處理中與其他表情符號被去除，所以在comment_text_basic_processing函式中預先替換為 '讚'

In [ ]:
# 檔案讀寫path
raw_restaurant_comments_folder = r"\專案實作\地區餐廳評論分析\google_restaurant_comments_txt\restaurant_comments_unprocessed"
revised_restaurant_comments_folder = r"\專案實作\地區餐廳評論分析\google_restaurant_comments_txt\restaurant_comments_processed"


def comment_text_basic_processing(data,col):
    """
    留言中表情符號字串去除
    特定表情符號轉換為對應意思中文詞彙
    :param data: 
    :param col: 
    :return: 
    """
    
    data[col] = data[col].str.replace('👍','讚')
    
    emoji_text = re.compile(
        r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F700-\U0001F77F'
        r'\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF'
        r'\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U0001FB00-\U0001FBFF'
        r'\U0001FC00-\U0001FCFF\U0001FD00-\U0001FDFF\U0001FE00-\U0001FEFF'
        r'\U0001FF00-\U0001FFFF\u2B05\u2B06\u2B07\u2934\u2935\u25AA\u25FE'
        r'\u2B05\u2B06\u2B07\u2934\u2935\u25AA\u25FE\u2600-\u26FF\u2700-\u27BF'
        r'\u2300-\u23FF\ufe0f]+',
        flags=re.UNICODE)
    data['評論留言原內容處理'] = data[col].apply(lambda x:emoji_text.sub('',x))


# ckiptagger 中文斷詞處理
# def segment_process(text):
#     """
#     評論留言斷詞
#     斷詞以 | 分隔
#     :param text: 
#     :return: 
#     """
#     content_text = ws([text])
#     return '|'.join(content_text[0])


# ckip Transformers nlp bert model 斷詞
def segment_process_2(text):
    """
    :param text: 
    :return: 
    """
    content_text = ckip_ws([text])
    return '|'.join(content_text[0])


def pos_process(text):
    content_text = ckip_pos([text])
    return  list(zip(text,content_text))


# def ner_process(text):
#     content_text = ckip_ner([text])
#     return  content_text


for root, dirs, files in os.walk(raw_restaurant_comments_folder):
    for dir in dirs:
        print("要處理的餐廳類別:",dir)
    for file_name in files:
        print("要處理的評論文字檔:",os.path.join(root,file_name))
    
        if file_name.endswith(".txt"):
            txt_file_path = os.path.join(root, file_name)
            file_name = os.path.basename(txt_file_path)
            # print(f"正在處理{file_name}評論文字檔...")
        
            # 創建對應同名餐廳類別資料夾
            sub_folder_name = os.path.basename(root)
            revised_sub_folder_path = os.path.join(revised_restaurant_comments_folder,sub_folder_name)
            os.makedirs(revised_sub_folder_path,exist_ok=True)
            # print(f"已創建{revised_sub_folder_path}資料夾")

            # 打開原始評論文字檔案轉換為dataframe
            try:
                with open(txt_file_path, 'r',encoding='utf-8') as file:
                    file_content = file.read()
                comments_str_list = file_content.split(',') # data type is list
                df_comments = pd.DataFrame(data=comments_str_list,columns=['評論留言原內容'])
                
                # 對評論留言原內容做基本處理存放於新欄位
                comment_text_basic_processing(df_comments,'評論留言原內容')
                df_comments['評論留言原內容處理'] = df_comments['評論留言原內容處理'].str.replace('[', '').str.replace(']', '').str.replace('\\n', '').str.replace("'", '').str.replace('\\u200d','')
                
                # 對評論留言原內容處理後進行斷詞處理
                df_comments['評論留言內容斷詞'] = df_comments['評論留言原內容處理'].apply(segment_process_2)
                
                # 對評論留言斷詞內容進行詞性標註
                # df_comments['評論留言詞性'] = df_comments['評論留言內容斷詞'].apply(pos_process)
                
                # 轉換為dataframe後輸出為excel檔案
                df_comments_file_name = os.path.splitext(file_name)[0] + '.xlsx'
                df_comments_file_path = os.path.join(revised_sub_folder_path,df_comments_file_name)
                df_comments.to_excel(df_comments_file_path,index=False)
                print(f"已將{txt_file_path} --> excel檔案，且轉存至{revised_sub_folder_path}")
                
            except Exception as e:
                print(f"轉換文件{txt_file_path}，出現錯誤:{str(e)}")
                

In [5]:
# 進行斷詞處理

In [6]:
# import os
# import glob
# import pandas as pd
# 
# # 設定主資料夾路徑
# main_folder = r'\專案實作\地區餐廳評論分析\google_restaurant_comments_txt\restaurant_comments_processed'
# 
# # 初始化變量用於存儲每個子資料夾的資料行數和Excel文件數量
# total_rows = 0
# total_files = 0
# 
# # 遍歷主資料夾中的所有子資料夾
# for folder in os.listdir(main_folder):
#     folder_path = os.path.join(main_folder, folder)
#     
#     # 檢查是否為資料夾
#     if os.path.isdir(folder_path):
#         # 在子資料夾中查找所有Excel文件
#         excel_files = glob.glob(os.path.join(folder_path, '*.xlsx')) + glob.glob(os.path.join(folder_path, '*.xls'))
#         
#         # 初始化變量用於存儲當前子資料夾的資料行數和Excel文件數量
#         folder_rows = 0
#         folder_files = len(excel_files)
#         
#         # 遍歷所有Excel文件
#         for excel_file in excel_files:
#             # 讀取Excel文件
#             df = pd.read_excel(excel_file)
#             
#             # 獲取資料行數
#             num_rows = df.shape[0]
#             
#             # 打印每個檔案的資料行數
#             print(f"檔案：{excel_file}，資料行數：{num_rows}")
#             
#             # 更新當前子資料夾的資料行數
#             folder_rows += num_rows
#         
#         # 更新總資料行數和總Excel文件數量
#         total_rows += folder_rows
#         total_files += folder_files
#         
#         # 計算當前子資料夾的平均資料行數
#         if folder_files > 0:
#             average_rows = folder_rows / folder_files
#             print(f"子資料夾：{folder}，平均資料行數：{average_rows}")
#         else:
#             print(f"子資料夾：{folder}，沒有Excel文件。")
# 
# # 計算所有子資料夾的平均資料行數
# if total_files > 0:
#     overall_average = total_rows / total_files
#     print(f"所有子資料夾的平均資料行數：{overall_average}")
# else:
#     print("沒有Excel文件。")


In [7]:
# from ckiptagger import data_utils, construct_dictionary, WS, POS
# 
# # 載入模型
# ws = WS("./data")
# pos = POS("./data")
# 
# # 設定停用詞
# stop_words = ['的', '了', '在', '是', '有', '我', '他', '她', '你', '妳']
# 
# # 處理文本
# text = "這是一段中文文本，我們要對它進行停用詞處理。"
# word_sentence_list = ckip_ws([text])
# pos_sentence_list = pos(word_sentence_list)
# 
# # 去除停用詞
# for i, sentence in enumerate(word_sentence_list):
#     for j, word in enumerate(sentence):
#         if word in stop_words:
#             word_sentence_list[i][j] = ''
# print(word_sentence_list)
